In [2]:
!pip install --upgrade pip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


Abbiamo messo su questo runtime la cartella "training" e la cartella "saved_models" e il file "requirements.txt".

Il file "requirements.txt" ha subito delle modifiche, in quanto non è possibile scaricare su colab delle librerie con il suffisso "+cu129". Il nuovo file è il seguente:

albucore==0.0.24
albumentations==2.0.8
annotated-types==0.7.0
colorama==0.4.6
contourpy==1.3.3
cycler==0.12.1
filelock==3.13.1
fonttools==4.59.2
fsspec==2024.6.1
Jinja2==3.1.4
joblib==1.5.2
kiwisolver==1.4.9
lightning-utilities==0.15.2
MarkupSafe==2.1.5
matplotlib==3.10.6
mpmath==1.3.0
networkx==3.3
numpy==2.1.2
opencv-python-headless==4.12.0.88
packaging==25.0
pillow==11.0.0
pycocotools==2.0.10
pydantic==2.11.7
pydantic_core==2.33.2
pyparsing==3.2.3
python-dateutil==2.9.0.post0
PyYAML==6.0.2
scikit-learn==1.7.2
scipy==1.16.1
setuptools==70.2.0
simsimd==6.5.3
six==1.17.0
stringzilla==3.12.6
sympy==1.13.3
threadpoolctl==3.6.0
torch==2.8.0
torchaudio==2.8.0
torchmetrics==1.8.2
torchvision==0.23.0
tqdm==4.67.1
typing-inspection==0.4.1
typing_extensions==4.12.2

Il codice di seguito ci permette di scaricare il dataset su Colab:

In [3]:
!pip install roboflow

from roboflow import Roboflow
import shutil
import os

rf = Roboflow(api_key="nUEeuvVZlZq0HQzkJe6e")
project = rf.workspace("objectdetection-uzld5").project("coco-home-objects")
version = project.version(8)
dataset = version.download("coco")

#per spostare la cartella scaricata in /training
dataset_path = dataset.location
print("Dataset scaricato in:", dataset_path)
#dove voglio spostare la cartella scaricata
target_dir = "/content/training"
# Sposta l'intera cartella nella directory "training"
destination_path = os.path.join(target_dir, os.path.basename(dataset_path))
shutil.move(dataset_path, destination_path)
print("Dataset spostato in:", destination_path)

#rinomina la cartella all'interno di "/training"
new_name = "COCO-Home-Objects"
new_path = os.path.join(target_dir, new_name)
os.rename(destination_path, new_path)
print("Cartella rinominata in:", new_path)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 86.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 39.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 118.2 MB/s  0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [roboflow]
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to COCO-Home-Objects-8 in coco:: 100%|██████████| 13935/13935 [00:03<00:00, 4013.66it/s]


Dataset scaricato in: /content/COCO-Home-Objects-8
Dataset spostato in: /content/training/COCO-Home-Objects-8
Cartella rinominata in: /content/training/COCO-Home-Objects


Abbiamo scaricato il dataset da Roboflow e lo sposta direttamente nella cartella /training/

Ora installiamo tutte le librerie presenti in "requirements.txt"

In [9]:
#Voglio scaricare tutte le librerie presenti in "requirements.txt"
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


Ora tutte le librerie richieste dovrebbero essere disponibili

Verifichiamo che prende correttamente la GPU

In [10]:
import torch
torch.cuda.is_available(), torch.cuda.get_device_name(0)

(True, 'Tesla T4')

Di seguito il file "main.py" per poter costruire il dataset e avviare il train del modello:

In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import json
import logging
import os
import math

#La root del progetto su colab si chiama "/content/"
#Per poter importare il contenuto della cartella /content/training:
import sys
sys.path.append('/content/training')
os.chdir('/content/training')
#Adesso possiamo iniziare il codice presente nel file "main.py"


from dataset_builder import CocoHomeDataset, get_train_transforms, get_val_transforms
from model_builder import create_ssd_model
from trainer_net import training_net
from tester_net import evaluate_model

# La tua funzione collate_fn
def collate_fn(batch):
    batch = list(filter(lambda x: x is not None, batch))
    if not batch: return None, None
    batch = list(filter(lambda x: x[1]['boxes'].numel() > 0, batch))
    if not batch: return None, None
    return tuple(zip(*batch))

def main():
    # Svuota la cache della GPU ---
    torch.cuda.empty_cache()

    #Configura la logica dei log
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', filename='training_log_ssd_coco.txt', filemode='a')
    with open('config.json', 'r') as f:
        config = json.load(f)

    #Seleziona il dispositivo cuda
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Utilizzo del dispositivo: {device}")
    logging.info(f"Dispositivo: {device}, Parametri: batch_size={config['batch_size']}, epochs={config['num_epochs']}, lr={config['learning_rate']}")

    #Configura i percorsi dei dataset con le relative annotations
    data_dir = config['data_dir']
    annotations_file_template = config['annotations_file']
    train_ann_file = os.path.join(data_dir, annotations_file_template.format('train'))
    train_img_dir = os.path.join(data_dir, 'train')
    val_ann_file = os.path.join(data_dir, annotations_file_template.format('valid'))
    val_img_dir = os.path.join(data_dir, 'valid')

    print("Caricamento dataset di training...")
    temp_dataset = CocoHomeDataset(images_dir=train_img_dir, annotations_file=train_ann_file)
    num_classes = temp_dataset.num_classes
    train_dataset = CocoHomeDataset(images_dir=train_img_dir, annotations_file=train_ann_file, transforms=get_train_transforms())
    val_dataset = CocoHomeDataset(images_dir=val_img_dir, annotations_file=val_ann_file, transforms=get_val_transforms())

    #Caricamento dei Dataloader
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=config['num_workers'], collate_fn=collate_fn, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=config['num_workers'], collate_fn=collate_fn)

    # --- Creazione del modello SSD ---
    print("Creazione del modello SSD...")
    model = create_ssd_model(num_classes=num_classes, device=device)

    # --- Strategia di addestramento a due fasi con Early Stopping ---

    # FASE 1: Addestramento della testa
    print("\n--- FASE 1: Congelamento del backbone e addestramento della testa ---")
    for param in model.backbone.parameters():
        param.requires_grad = False
    params_head_only = [p for p in model.parameters() if p.requires_grad]
    optimizer = optim.AdamW(params_head_only, lr=config['learning_rate'])
    total_epochs = config['num_epochs']
    head_train_percent = config.get('head_train_percent', 0.2)
    head_epochs = math.ceil(total_epochs * head_train_percent)
    print(f"Inizio addestramento della sola testa per {head_epochs} epoche...")
    training_net(model, train_loader, num_epochs=head_epochs, device=device, optimizer=optimizer, lr_scheduler=None)

    # FASE 2: Fine-tuning con Early Stopping
    remaining_epochs = total_epochs - head_epochs
    print(f"\n--- FASE 2: Scongelamento e fine-tuning completo per un massimo di {remaining_epochs} epoche ---")

    for param in model.parameters():
        param.requires_grad = True

    optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'] / 100, weight_decay=0.05)

    if remaining_epochs > 0:
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=remaining_epochs)
    else:
        lr_scheduler = None

    # Inizializzazione per l'Early Stopping
    best_map = 0.0
    patience_limit = 10
    patience_counter = 0

    if remaining_epochs > 0:
        for epoch in range(remaining_epochs):
            print(f"\n--- Inizio Fine-Tuning: Epoca {epoch + 1}/{remaining_epochs} ---")

            model = training_net(model, train_loader, num_epochs=1, device=device, optimizer=optimizer, lr_scheduler=lr_scheduler)
            results = evaluate_model(model, val_loader, device)

            if results is None or 'map' not in results:
                print("Valutazione fallita, continuo con la prossima epoca.")
                continue

            current_map = results['map'].item()

            if current_map > best_map:
                best_map = current_map
                patience_counter = 0
                print(f"Nuovo mAP migliore! {best_map:.4f}. Salvo il modello in 'best_model.pth'.")
                torch.save(model.state_dict(), 'best_model.pth')
                logging.info(f"Nuovo mAP migliore: {best_map:.4f} all'epoca {epoch + 1} del fine-tuning.")
            else:
                patience_counter += 1
                print(f"Nessun miglioramento del mAP. Pazienza: {patience_counter}/{patience_limit}")

            if patience_counter >= patience_limit:
                print(f"Early stopping: le performance non migliorano da {patience_limit} epoche. Interrompo l'addestramento.")
                logging.info(f"Early stopping attivato. mAP migliore raggiunto: {best_map:.4f}")
                break

    print(f"\nAddestramento completato. Il miglior mAP ottenuto è stato: {best_map:.4f}")

    # Carichiamo il modello migliore per sicurezza, anche se l'ultimo salvataggio dovrebbe essere quello
    print("Caricamento del modello con le migliori performance...")
    model.load_state_dict(torch.load('best_model.pth'))

    # Valutazione finale sul modello migliore
    print("\n--- Valutazione finale sul modello migliore ---")
    evaluate_model(model, val_loader, device)

    print("Processo completato.")

if __name__ == '__main__':
    main()

Utilizzo del dispositivo: cuda
Caricamento dataset di training...
loading annotations into memory...
Done (t=1.05s)
creating index...
index created!
loading annotations into memory...
Done (t=1.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
Creazione del modello SSD...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(



--- FASE 1: Congelamento del backbone e addestramento della testa ---
Inizio addestramento della sola testa per 20 epoche...

Inizio addestramento...


Epoca [1/20]: 100%|██████████| 3213/3213 [04:41<00:00, 11.40it/s, loss=16.5041]


Fine Epoca [1/20], Perdita media: 14.2573


...

Epoca [1/1]: 100%|██████████| 3213/3213 [06:37<00:00,  8.08it/s, loss=6.9639]

Fine Epoca [1/1], Perdita media: 3.5653

Inizio fase di valutazione...


Valutazione completata. Risultati mAP Dettagliati: {'map': tensor(0.2233), 'map_50': tensor(0.3838), 'map_75': tensor(0.2379), 'map_small': tensor(0.0457), 'map_medium': tensor(0.1668), 'map_large': tensor(0.3669), 'mar_1': tensor(0.2279), 'mar_10': tensor(0.3132), 'mar_100': tensor(0.3293), 'mar_small': tensor(0.0838), 'mar_medium': tensor(0.2831), 'mar_large': tensor(0.4718), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,
         16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,
         30,  31,  32,  33,  34,  36,  37,  38,  39,  41,  42,  43,  44,  46,
         47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,
         61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  74,  75,
         76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
         90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103

Epoca [1/1]:  84%|████████▍ | 2703/3213 [05:35<01:03,  8.05it/s, loss=2.4851]


KeyboardInterrupt: 

------ ATTENZIONE: ESEGUIRE QUESTO CODICE SOLO NEL CASO IN CUI SI DEVE RIPARTIRE DA UNA CERTA EPOCA NELLA FASE 2 ---------------------------

Di seguito il codice del Main modificato per poter ripartire dalla Fase 2 e in un determinato momento di addestramento (numero di epoca su 80 totali rimanenti), con relativo numero di pazienza:
ATTENZIONE: ESEGUIRE QUESTO CODICE SOLO NEL CASO IN CUI NON SI DISPONE  Di seguito il codice del Main modificato per poter ripartire dalla Fase 2 e in un determinato momento di addestramento (numero di epoca su 80 totali rimanenti), con relativo numero di pazienza:

In [12]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import json
import logging
import os
import math

#La root del progetto su colab si chiama "/content/"
#Per poter importare il contenuto della cartella /content/training:
import sys
sys.path.append('/content/training')
os.chdir('/content/training')
#Adesso possiamo iniziare il codice presente nel file "main.py"


from dataset_builder import CocoHomeDataset, get_train_transforms, get_val_transforms
from model_builder import create_ssd_model
from trainer_net import training_net
from tester_net import evaluate_model

# La tua funzione collate_fn
def collate_fn(batch):
    batch = list(filter(lambda x: x is not None, batch))
    if not batch: return None, None
    batch = list(filter(lambda x: x[1]['boxes'].numel() > 0, batch))
    if not batch: return None, None
    return tuple(zip(*batch))

def main():
    # Svuota la cache della GPU ---
    torch.cuda.empty_cache()

    #Configura la logica dei log
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', filename='training_log_ssd_coco.txt', filemode='a')
    with open('config.json', 'r') as f:
        config = json.load(f)

    #Seleziona il dispositivo cuda
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Utilizzo del dispositivo: {device}")
    logging.info(f"Dispositivo: {device}, Parametri: batch_size={config['batch_size']}, epochs={config['num_epochs']}, lr={config['learning_rate']}")

    #Configura i percorsi dei dataset con le relative annotations
    data_dir = config['data_dir']
    annotations_file_template = config['annotations_file']
    train_ann_file = os.path.join(data_dir, annotations_file_template.format('train'))
    train_img_dir = os.path.join(data_dir, 'train')
    val_ann_file = os.path.join(data_dir, annotations_file_template.format('valid'))
    val_img_dir = os.path.join(data_dir, 'valid')

    print("Caricamento dataset di training...")
    temp_dataset = CocoHomeDataset(images_dir=train_img_dir, annotations_file=train_ann_file)
    num_classes = temp_dataset.num_classes
    train_dataset = CocoHomeDataset(images_dir=train_img_dir, annotations_file=train_ann_file, transforms=get_train_transforms())
    val_dataset = CocoHomeDataset(images_dir=val_img_dir, annotations_file=val_ann_file, transforms=get_val_transforms())

    #Caricamento dei Dataloader
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=config['num_workers'], collate_fn=collate_fn, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=config['num_workers'], collate_fn=collate_fn)

    # --- Creazione del modello SSD ---
    print("Creazione del modello SSD...")
    model = create_ssd_model(num_classes=num_classes, device=device)

    # --- Strategia di addestramento a due fasi con Early Stopping ---

    # FASE 1: Addestramento della testa
    # print("\n--- FASE 1: Congelamento del backbone e addestramento della testa ---")
    '''for param in model.backbone.parameters():
        param.requires_grad = False
    params_head_only = [p for p in model.parameters() if p.requires_grad]
    optimizer = optim.AdamW(params_head_only, lr=config['learning_rate'])
    total_epochs = config['num_epochs']
    head_train_percent = config.get('head_train_percent', 0.2)
    head_epochs = math.ceil(total_epochs * head_train_percent)
    print(f"Inizio addestramento della sola testa per {head_epochs} epoche...")
    training_net(model, train_loader, num_epochs=head_epochs, device=device, optimizer=optimizer, lr_scheduler=None)
    '''

    # FASE 2: Fine-tuning con Early Stopping
    total_epochs = config['num_epochs']
    head_epochs = 20
    epoche_svolte = 18 # NUMERO CHE VA CAMBIATO SULLA BASE DEI PESI DEL MODELLO SALVATO
    remaining_epochs = total_epochs - head_epochs - epoche_svolte

    #model = create_ssd_model(num_classes=num_classes, device=device) # Assicurati che num_classes e device siano definiti
    model.load_state_dict(torch.load('best_model_18_80_pazienza_4_10.pth')) # DA MODIFICARE PRIMA DI ESEGUIRE
    print("Pesi del modello caricati con successo!")

    print(f"\n--- FASE 2: Scongelamento e fine-tuning completo per un massimo di {remaining_epochs} epoche ---")

    for param in model.parameters():
        param.requires_grad = True

    optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'] / 100, weight_decay=0.05)

    if remaining_epochs > 0:
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=remaining_epochs)
    else:
        lr_scheduler = None

    # Inizializzazione per l'Early Stopping
    best_map = 0.0
    patience_limit = 10
    patience_counter = 0

    if remaining_epochs > 0:
        for epoch in range(remaining_epochs):
            print(f"\n--- Inizio Fine-Tuning: Epoca {epoch + 1}/{remaining_epochs} ---")

            model = training_net(model, train_loader, num_epochs=1, device=device, optimizer=optimizer, lr_scheduler=lr_scheduler)
            results = evaluate_model(model, val_loader, device)

            if results is None or 'map' not in results:
                print("Valutazione fallita, continuo con la prossima epoca.")
                continue

            current_map = results['map'].item()

            if current_map > best_map:
                best_map = current_map
                patience_counter = 0
                print(f"Nuovo mAP migliore! {best_map:.4f}. Salvo il modello in 'best_model.pth'.")
                torch.save(model.state_dict(), 'best_model.pth')
                logging.info(f"Nuovo mAP migliore: {best_map:.4f} all'epoca {epoch + 1} del fine-tuning.")
            else:
                patience_counter += 1
                print(f"Nessun miglioramento del mAP. Pazienza: {patience_counter}/{patience_limit}")

            if patience_counter >= patience_limit:
                print(f"Early stopping: le performance non migliorano da {patience_limit} epoche. Interrompo l'addestramento.")
                logging.info(f"Early stopping attivato. mAP migliore raggiunto: {best_map:.4f}")
                break

    print(f"\nAddestramento completato. Il miglior mAP ottenuto è stato: {best_map:.4f}")

    # Carichiamo il modello migliore per sicurezza, anche se l'ultimo salvataggio dovrebbe essere quello
    print("Caricamento del modello con le migliori performance...")
    model.load_state_dict(torch.load('best_model.pth'))

    # Valutazione finale sul modello migliore
    print("\n--- Valutazione finale sul modello migliore ---")
    evaluate_model(model, val_loader, device)

    print("Processo completato.")

if __name__ == '__main__':
    main()

Utilizzo del dispositivo: cuda
Caricamento dataset di training...
loading annotations into memory...
Done (t=1.01s)
creating index...
index created!
loading annotations into memory...
Done (t=1.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
Creazione del modello SSD...
Pesi del modello caricati con successo!

--- FASE 2: Scongelamento e fine-tuning completo per un massimo di 62 epoche ---

--- Inizio Fine-Tuning: Epoca 1/62 ---

Inizio addestramento...


Epoca [1/1]:   3%|▎         | 84/3213 [00:13<08:31,  6.12it/s, loss=4.2647]


KeyboardInterrupt: 

In [ ]:
### APPUNTI PER SALVARE PESI E/O MODELLO
'''
# Salva solo i pesi
torch.save(model.state_dict(), 'pesi.pt')
print("Pesi del modello salvati con successo!")

# --- Per caricare i pesi ---
model2 = create_ssd_model(num_classes=num_classes, device=device) # Assicurati che num_classes e device siano definiti
model2.load_state_dict(torch.load('pesi.pt'))
print("Pesi del modello caricati con successo!")

#-----------------------------------------------------------

# Salva l’intero modello
torch.save(model, 'modello_completo.pt')

# --- Per caricare il modello ---
model3 = torch.load('modello_completo.pt')
'''